In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('processed_data.csv')

In [2]:
patient_counts = data['patient_id'].value_counts()
patient_counts_pd = pd.DataFrame(patient_counts)
patient_counts_pd = patient_counts_pd.loc[patient_counts_pd['count'] > 29]

filtered_data = data[data['patient_id'].isin(patient_counts_pd.index)]  # 해당 ID만 남기기
filtered_data.to_csv('filtered_id_data.csv')

In [3]:
threshold_high = 25.34 

threshold_low = -2.61

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

labels = ['bad', 'mid', 'good']

for pid, group in filtered_data.groupby('patient_id'):

    # feature / target 나누기
    y = group['score_level']
    X = group.drop(columns=['sleep_quality_score', 'score_level', 'patient_id', 'timestamp'])
    

    # train/test split (시계열 고려해서 shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )


    # 모델 정의
    clf = DecisionTreeClassifier(max_depth=6, random_state=42)
    clf.fit(X_train, y_train)

    # 예측
    y_pred = clf.predict(X_test)

    # 평가
    print(confusion_matrix(y_test, y_pred, labels=labels))
    print(classification_report(y_test, y_pred, labels=labels))



[[ 1  1  1]
 [ 8 71  4]
 [ 2  9  6]]
              precision    recall  f1-score   support

         bad       0.09      0.33      0.14         3
         mid       0.88      0.86      0.87        83
        good       0.55      0.35      0.43        17

    accuracy                           0.76       103
   macro avg       0.50      0.51      0.48       103
weighted avg       0.80      0.76      0.77       103

[[ 1  7 20]
 [ 5 51 60]
 [ 1 12 76]]
              precision    recall  f1-score   support

         bad       0.14      0.04      0.06        28
         mid       0.73      0.44      0.55       116
        good       0.49      0.85      0.62        89

    accuracy                           0.55       233
   macro avg       0.45      0.44      0.41       233
weighted avg       0.57      0.55      0.52       233

[[ 10  13   8]
 [ 19  45  39]
 [  1  58 111]]
              precision    recall  f1-score   support

         bad       0.33      0.32      0.33        31
         

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

              precision    recall  f1-score   support

         bad       0.66      0.55      0.60        77
         mid       0.56      0.73      0.64        79
        good       0.12      0.07      0.09        27

    accuracy                           0.56       183
   macro avg       0.45      0.45      0.44       183
weighted avg       0.54      0.56      0.54       183

[[ 4  4  3]
 [ 6 45  4]
 [ 5 18  5]]
              precision    recall  f1-score   support

         bad       0.27      0.36      0.31        11
         mid       0.67      0.82      0.74        55
        good       0.42      0.18      0.25        28

    accuracy                           0.57        94
   macro avg       0.45      0.45      0.43        94
weighted avg       0.55      0.57      0.54        94

[[ 9  6  2]
 [ 7 70  8]
 [ 6 14 15]]
              precision    recall  f1-score   support

         bad       0.41      0.53      0.46        17
         mid       0.78      0.82      0.80        85
 

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

[[1 3 1]
 [2 5 1]
 [0 0 0]]
              precision    recall  f1-score   support

         bad       0.33      0.20      0.25         5
         mid       0.62      0.62      0.62         8
        good       0.00      0.00      0.00         0

    accuracy                           0.46        13
   macro avg       0.32      0.27      0.29        13
weighted avg       0.51      0.46      0.48        13

[[3 1 3]
 [1 5 1]
 [0 2 0]]
              precision    recall  f1-score   support

         bad       0.75      0.43      0.55         7
         mid       0.62      0.71      0.67         7
        good       0.00      0.00      0.00         2

    accuracy                           0.50        16
   macro avg       0.46      0.38      0.40        16
weighted avg       0.60      0.50      0.53        16

[[96  3  2]
 [ 4 71  0]
 [ 0  0  0]]
              precision    recall  f1-score   support

         bad       0.96      0.95      0.96       101
         mid       0.96      0.95   

...

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

              precision    recall  f1-score   support

         bad       0.56      0.28      0.37        18
         mid       0.38      0.47      0.42        17
        good       0.00      0.00      0.00         2

    accuracy                           0.35        37
   macro avg       0.31      0.25      0.26        37
weighted avg       0.45      0.35      0.37        37

[[ 0  5  1]
 [ 0 29  8]
 [ 0 10  1]]
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         6
         mid       0.66      0.78      0.72        37
        good       0.10      0.09      0.10        11

    accuracy                           0.56        54
   macro avg       0.25      0.29      0.27        54
weighted avg       0.47      0.56      0.51        54

[[ 0  0  0]
 [ 0 56  0]
 [ 0  0  0]]
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         0
         mid       1.00      1.00      1.00        56
 

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         2
         mid       0.74      1.00      0.85        17
        good       0.00      0.00      0.00         5

    accuracy                           0.71        24
   macro avg       0.25      0.33      0.28        24
weighted avg       0.52      0.71      0.60        24

[[ 0  0  0]
 [ 0 62  0]
 [ 0  0  0]]
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         0
         mid       1.00      1.00      1.00        62
        good       0.00      0.00      0.00         0

    accuracy                           1.00        62
   macro avg       0.33      0.33      0.33        62
weighted avg       1.00      1.00      1.00        62

[[ 2  6  4]
 [ 2 14  9]
 [ 2 10  1]]
              precision    recall  f1-score   support

         bad       0.33      0.17      0.22        12
         mid       0.47      0.56      0.51        25
 

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is